In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import seaborn as sns

## Data Load

In [2]:
train = pd.read_csv('./book_train.csv')
test = pd.read_csv('./book_test.csv')
#sample = pd.read_csv('./submission_sample.csv')

In [3]:
result = test[['seq']]

In [4]:
test['is_canceled'] = 3
data = pd.concat([train, test], sort = False).set_index('seq')
data.head()

,is_canceled,hotel_type,lead_time,year,month,week_number,day,stays_in_weekend,stays_in_week,adults,...,pre_cancel,pre_not_cancel,reserved_room_type,assigned_room_type,booking_changes,deposit,days_in_basket,customer_type,required_parking,number_of_requests
seq,,,,,,,,,,,,,,,,,,,,,
B029399,0,R,7,2015,July,27,1,0,1,1,...,0,0,A,C,0,No Deposit,0,Transient,0,0
B029245,0,R,14,2015,July,27,1,0,2,2,...,0,0,A,A,0,No Deposit,0,Transient,0,1
B055775,0,R,14,2015,July,27,1,0,2,2,...,0,0,A,A,0,No Deposit,0,Transient,0,1
B093241,0,R,0,2015,July,27,1,0,2,2,...,0,0,C,C,0,No Deposit,0,Transient,0,0
B061800,0,R,9,2015,July,27,1,0,2,2,...,0,0,C,C,0,No Deposit,0,Transient,0,1


In [5]:
# data child의 결측치
data['child'] = data['child'].fillna(0)
data['child'] = data['child'].astype(int)

## Data Cleansing_Train

In [6]:
# hotel type
data['hotel_type_C'] = data['hotel_type'].apply(lambda x:1 if x == "C" else 0)
data['hotel_type_R'] = data['hotel_type'].apply(lambda x:1 if x == "R" else 0)
data = data.drop('hotel_type', axis = 1)

In [7]:
#lead_time
data['lead_time_0'] = data['lead_time'].apply(lambda x: 1 if x > 0 else 0)
data['lead_time_30'] = data['lead_time'].apply(lambda x: 1 if x >= 30 else 0)
data['lead_time_90'] = data['lead_time'].apply(lambda x: 1 if x >= 90 else 0)

In [8]:
# year
data['y_2015'] = data['year'].apply(lambda x: 1 if x == 2015 else 0)
data['y_2016'] = data['year'].apply(lambda x: 1 if x == 2016 else 0)
data['y_2017'] = data['year'].apply(lambda x: 1 if x == 2017 else 0)

In [9]:
#month --> 계절에 따라
data['spring'] = data['month'].apply(lambda x: 1 if x == 'March' or x == 'April' or x == 'May' else 0)
data['summer'] = data['month'].apply(lambda x: 1 if x == 'June' or x == 'July' or x == 'August' else 0)
data['fall'] = data['month'].apply(lambda x: 1 if x == 'September' or x == 'October' or x == 'November' else 0)
data['winter'] = data['month'].apply(lambda x: 1 if x == 'December' or x == 'January' or x == 'February' else 0)

In [10]:
#day
data['day_0'] = data['day'].apply(lambda x: 1 if x <= 10 else 0)
data['day_10'] = data['day'].apply(lambda x: 1 if x >= 11 and x <= 20 else 0)
data['day_20'] = data['day'].apply(lambda x: 1 if x >= 21 else 0)

In [11]:
def filter_month(x):
    if x == 'January':
        return 1
    elif x == 'February':
        return 2
    elif x == 'March':
        return 3
    elif x == 'April':
        return 4
    elif x == 'May':
        return 5
    elif x == 'June':
        return 6
    elif x == 'July':
        return 7
    elif x == 'August':
        return 8
    elif x == 'September':
        return 9
    elif x == 'October':
        return 10
    elif x == 'November':
        return 11
    elif x == 'December':
        return 12

In [12]:
# 월을 숫자로
data['month'] = data['month'].apply(lambda x: filter_month(x))

#형식 변환
data['year'] = data['year'].astype(str)
data['month'] = data['month'].astype(str)
data['day'] = data['day'].astype(str)

#형식 변환2
data['month'] = data['month'].apply(lambda x: x.zfill(2))
data['day'] = data['day'].apply(lambda x: x.zfill(2))

In [13]:
# datetime으로 변환
data['date'] = data['year']+data['month']+data['day']
data['date'] = pd.to_datetime(data['date'], format = '%Y%m%d')

#요일 --> 숫자
data['weekdays'] = data['date'].apply(lambda x: x.weekday())

In [14]:
data['weekdays'] = data['date'].apply(lambda x: x.weekday())

In [15]:
#각 요일
data['w_Mon'] = data['weekdays'].apply(lambda x: 1 if x == 0 else 0)
data['w_Tue'] = data['weekdays'].apply(lambda x: 1 if x == 1 else 0)
data['w_Wed'] = data['weekdays'].apply(lambda x: 1 if x == 2 else 0)
data['w_Thu'] = data['weekdays'].apply(lambda x: 1 if x == 3 else 0)
data['w_Fri'] = data['weekdays'].apply(lambda x: 1 if x == 4 else 0)
data['w_Sat'] = data['weekdays'].apply(lambda x: 1 if x == 5 else 0)
data['w_Sun'] = data['weekdays'].apply(lambda x: 1 if x == 6 else 0)

#주말 유뮤
data['w_weekends'] = data['weekdays'].apply(lambda x: 1 if x == 5 or x == 6 else 0)

In [16]:
# year, month, day값 변환
data = data.drop(['year', 'month', 'date', 'weekdays'], axis =1 )
data['day'] = data['day'].astype(int)

In [17]:
#week_number
data['week0_13'] = data['week_number'].apply(lambda x: 1 if x >= 0 and x <= 13 else 0)
data['week14_26'] = data['week_number'].apply(lambda x: 1 if x >= 14 and x <= 26 else 0)
data['week27_39'] = data['week_number'].apply(lambda x: 1 if x >= 27 and x <= 39 else 0)
data['week39_53'] = data['week_number'].apply(lambda x: 1 if x >= 39 and x <= 53 else 0)

In [18]:
# stays_in_weekend
data['stays_in_weekend_Y'] = data['stays_in_weekend'].apply(lambda x: 1 if x > 0 else 0)
data['st_weekend0'] = data['stays_in_weekend'].apply(lambda x: 1 if x == 0 else 0)

data['st_weekend1'] = data['stays_in_weekend'].apply(lambda x: 1 if x == 1 else 0)
data['st_weekend2'] = data['stays_in_weekend'].apply(lambda x: 1 if x == 2 else 0)
data['st_weekend3'] = data['stays_in_weekend'].apply(lambda x: 1 if x == 3 else 0)
data['st_weekend4'] = data['stays_in_weekend'].apply(lambda x: 1 if x == 4 else 0)
data['st_weekend5'] = data['stays_in_weekend'].apply(lambda x: 1 if x >= 5 else 0)

In [19]:
# stays_in_weekend
data['stays_in_week_Y'] = data['stays_in_week'].apply(lambda x: 1 if x > 0 else 0)
data['st_week0'] = data['stays_in_weekend'].apply(lambda x: 1 if x == 0 else 0)

data['st_week1'] = data['stays_in_week'].apply(lambda x: 1 if x == 1 else 0)
data['st_week2'] = data['stays_in_week'].apply(lambda x: 1 if x == 2 else 0)
data['st_week3'] = data['stays_in_week'].apply(lambda x: 1 if x == 3 else 0)
data['st_week4'] = data['stays_in_week'].apply(lambda x: 1 if x == 4 else 0)
data['st_week5'] = data['stays_in_week'].apply(lambda x: 1 if x == 5 else 0)

data['st_week6'] = data['stays_in_week'].apply(lambda x: 1 if x == 6 else 0)
data['st_week7'] = data['stays_in_week'].apply(lambda x: 1 if x == 7 else 0)
data['st_week8'] = data['stays_in_week'].apply(lambda x: 1 if x == 8 else 0)
data['st_week9'] = data['stays_in_week'].apply(lambda x: 1 if x == 9 else 0)
data['st_week10'] = data['stays_in_week'].apply(lambda x: 1 if x == 10 else 0)

data['st_week11'] = data['stays_in_week'].apply(lambda x: 1 if x == 11 else 0)
data['st_week12'] = data['stays_in_week'].apply(lambda x: 1 if x == 12 else 0)
data['st_weeek13'] = data['stays_in_week'].apply(lambda x: 1 if x == 13 else 0)
data['st_week14'] = data['stays_in_week'].apply(lambda x: 1 if x == 14 else 0)
data['st_week15'] = data['stays_in_week'].apply(lambda x: 1 if x == 15 else 0)

data['st_week16'] = data['stays_in_week'].apply(lambda x: 1 if x >= 16 else 0)

In [20]:
#adults, child, baby
data['child_Y'] = data['child'].apply(lambda x: 1 if x > 0 else 0)
data['baby_Y'] = data['baby'].apply(lambda x: 1 if x > 0 else 0)
data['ch&ba_Y'] = data.apply(lambda x: 1 if x['child_Y'] == 1 or x['baby_Y'] == 1 else 0, axis = 1)

In [21]:
#meal
data['meal_BB'] = data['meal'].apply(lambda x:1 if x == "BB" else 0)
data['meal_HB'] = data['meal'].apply(lambda x:1 if x == "HB" else 0)
data['meal_FB'] = data['meal'].apply(lambda x:1 if x == "FB" else 0)
data['meal_NO'] = data['meal'].apply(lambda x:1 if x == "No_meal" else 0)
data = data.drop('meal', axis = 1)

In [22]:
#country
a = data.query('is_canceled != 3')[['is_canceled', 'country']]

num_country = a['country'].value_counts().reset_index()
num_country.rename(columns = {"index": "country",
                              "country": "sum"},
                   inplace=True)
can_country = a.query('is_canceled == 1').groupby('country')['is_canceled'].sum().reset_index()

df_cnt = pd.merge(num_country, can_country, on = 'country', how = 'left')
df_cnt['is_canceled'] = df_cnt['is_canceled'].fillna(0)

df_cnt['c_prob'] = round(df_cnt['is_canceled'] / df_cnt['sum'], 4)
fin_cnt = df_cnt[['country','c_prob']]

data = pd.merge(data, fin_cnt, on = 'country', how = 'left')
data['c_prob'] = data['c_prob'].fillna(0)

In [23]:
data['c_PRT'] = data['country'].apply(lambda x: 1 if x == 'PRT' else 0 )
data['c_GBR'] = data['country'].apply(lambda x: 1 if x == 'GBR' else 0 )
data['c_FRA'] = data['country'].apply(lambda x: 1 if x == 'FRA' else 0 )
data['c_ESP'] = data['country'].apply(lambda x: 1 if x == 'ESP' else 0 )
data['c_DEU'] = data['country'].apply(lambda x: 1 if x == 'DEU' else 0 )

data['c_ITA'] = data['country'].apply(lambda x: 1 if x == 'ITA' else 0 )
data['c_IRL'] = data['country'].apply(lambda x: 1 if x == 'IRL' else 0 )
data['c_BEL'] = data['country'].apply(lambda x: 1 if x == 'BEL' else 0 )
data['c_BRA'] = data['country'].apply(lambda x: 1 if x == 'BRA' else 0 )
data['c_NLD'] = data['country'].apply(lambda x: 1 if x == 'NLD' else 0 )

data['c_USA'] = data['country'].apply(lambda x: 1 if x == 'USA' else 0 )
data['c_CHE'] = data['country'].apply(lambda x: 1 if x == 'CHE' else 0 )
data['c_AUT'] = data['country'].apply(lambda x: 1 if x == 'AUT' else 0 )
data['c_CN'] = data['country'].apply(lambda x: 1 if x == 'CN' else 0 )
data['c_SWE'] = data['country'].apply(lambda x: 1 if x == 'SWE' else 0 )

data['c_CHN'] = data['country'].apply(lambda x: 1 if x == 'CHN' else 0 )

In [24]:
data = data.drop('country', axis = 1)

In [25]:
#book_method
data['book_on'] = data['book_method'].apply(lambda x: 1 if x == 'Online TA' else 0)
data['book_off'] = data['book_method'].apply(lambda x: 1 if x == 'Offline TA/TO' else 0)
data['book_grp'] = data['book_method'].apply(lambda x: 1 if x == 'Groups' else 0)
data['book_dir'] = data['book_method'].apply(lambda x: 1 if x == 'Direct' else 0)
data['book_cor'] = data['book_method'].apply(lambda x: 1 if x == 'Corporate' else 0)
data['book_oth'] = data['book_method'].apply(lambda x: 1 if x == 'Others' else 0)
data = data.drop('book_method', axis = 1)

In [26]:
#is_repeated_guest

In [27]:
#pre_cancel: 이전에 취소 횟수
data['pre_cancel_0'] = data['pre_cancel'].apply(lambda x: 1 if x == 0 else 1)
data['pre_cancel_1'] = data['pre_cancel'].apply(lambda x: 1 if x == 1 else 1)
data['pre_cancel_2'] = data['pre_cancel'].apply(lambda x: 2 if x == 2 else 1)

data['pre_cancel_1_10'] = data['pre_cancel'].apply(lambda x: 1 if x >= 3 and x < 10 else 1)

data['pre_cancel_10'] = data['pre_cancel'].apply(lambda x: 1 if x >= 10 else 1)

In [28]:
#pre_not_cancel
data['pre_not_cancel_0'] = data['pre_not_cancel'].apply(lambda x: 1 if x == 0 else 1)

data['pre_not_cancel_1'] = data['pre_not_cancel'].apply(lambda x: 1 if x == 1 else 1)
data['pre_not_cancel_2'] = data['pre_not_cancel'].apply(lambda x: 1 if x == 2 else 1)
data['pre_not_cancel_3'] = data['pre_not_cancel'].apply(lambda x: 1 if x == 3 else 1)
data['pre_not_cancel_4'] = data['pre_not_cancel'].apply(lambda x: 1 if x == 4 else 1)
data['pre_not_cancel_5'] = data['pre_not_cancel'].apply(lambda x: 1 if x == 5 else 1)

data['pre_not_cancel_6'] = data['pre_not_cancel'].apply(lambda x: 1 if x == 6 else 1)
data['pre_not_cancel_7'] = data['pre_not_cancel'].apply(lambda x: 1 if x >= 7 else 1)

In [29]:
# pre_cancel & pre_not_cancel
data['canceled_neg']  = (data['pre_not_cancel'] - data['pre_cancel']).apply(lambda x:1 if x < 0 else 0 )
data['canceled_0']  = (data['pre_not_cancel'] - data['pre_cancel']).apply(lambda x:1 if x ==  0 else 0 )
data['canceled_pos']  = (data['pre_not_cancel'] - data['pre_cancel']).apply(lambda x:1 if x > 0 else 0 )

In [30]:
# reserved_room_tpye & assigned_room_type
data['rm_change'] = data.apply(lambda x: 1 if x['reserved_room_type'] != x['assigned_room_type'] else 0, axis = 1)

In [31]:
# reserved_room_type
data['R_room_ty_A'] =data['reserved_room_type'].apply(lambda x:1 if x == "A" else 0)
data['R_room_ty_B'] = data['reserved_room_type'].apply(lambda x:1 if x == "B" else 0)
data['R_room_ty_C'] = data['reserved_room_type'].apply(lambda x:1 if x == "C" else 0)
data['R_room_ty_D'] = data['reserved_room_type'].apply(lambda x:1 if x == "D" else 0)
data['R_room_ty_E'] = data['reserved_room_type'].apply(lambda x:1 if x == "E" else 0)
data['R_room_ty_F'] = data['reserved_room_type'].apply(lambda x:1 if x == "F" else 0)
data['R_room_ty_G'] = data['reserved_room_type'].apply(lambda x:1 if x == "G" else 0)
data['R_room_ty_H'] = data['reserved_room_type'].apply(lambda x:1 if x == "H" else 0)
data['R_room_ty_P'] = data['reserved_room_type'].apply(lambda x:1 if x == "P" else 0)
data['R_room_ty_L'] = data['reserved_room_type'].apply(lambda x:1 if x == "L" else 0)
data = data.drop('reserved_room_type', axis = 1)

In [32]:
# assigned_room_type
data['A_room_ty_A'] = data['assigned_room_type'].apply(lambda x:1 if x == "A" else 0)
data['A_room_ty_B'] = data['assigned_room_type'].apply(lambda x:1 if x == "B" else 0)
data['A_room_ty_C'] = data['assigned_room_type'].apply(lambda x:1 if x == "C" else 0)
data['A_room_ty_D'] = data['assigned_room_type'].apply(lambda x:1 if x == "D" else 0)
data['A_room_ty_E'] = data['assigned_room_type'].apply(lambda x:1 if x == "E" else 0)
data['A_room_ty_F'] = data['assigned_room_type'].apply(lambda x:1 if x == "F" else 0)
data['A_room_ty_G'] = data['assigned_room_type'].apply(lambda x:1 if x == "G" else 0)
data['A_room_ty_H'] = data['assigned_room_type'].apply(lambda x:1 if x == "H" else 0)
data['A_room_ty_I'] = data['assigned_room_type'].apply(lambda x:1 if x == "I" else 0)
data['A_room_ty_K'] = data['assigned_room_type'].apply(lambda x:1 if x == "K" else 0)
data['A_room_ty_P'] = data['assigned_room_type'].apply(lambda x:1 if x == "P" else 0)

data = data.drop('assigned_room_type', axis = 1)

In [33]:
# booking_change
data['booking_ch_0'] = data['booking_changes'].apply(lambda x: 1 if x == 0 else 0)

data['booking_ch_1'] = data['booking_changes'].apply(lambda x: 1 if x == 1 else 0)
data['booking_ch_2'] = data['booking_changes'].apply(lambda x: 1 if x == 2 else 0)
data['booking_ch_3'] = data['booking_changes'].apply(lambda x: 1 if x == 3 else 0)
data['booking_ch_4'] = data['booking_changes'].apply(lambda x: 1 if x == 4 else 0)
data['booking_ch_5'] = data['booking_changes'].apply(lambda x: 1 if x == 5 else 0)

data['booking_ch_6'] = data['booking_changes'].apply(lambda x: 1 if x >= 6 else 0)

In [34]:
#deposit
data['No_Dp'] = data['deposit'].apply(lambda x: 1 if x== 'No Deposit' else 0)
data['Non_Rf'] = data['deposit'].apply(lambda x: 1 if x== 'Non_Refund' else 0)
data['Rf'] = data['deposit'].apply(lambda x: 1 if x== 'Refundable' else 0)
data = data.drop('deposit', axis = 1)

In [35]:
#days_in_basket
data['basket_N'] = data['days_in_basket'].apply(lambda x: 1 if x == 0 else 0)

In [36]:
data['basket_0110'] = data['days_in_basket'].apply(lambda x: 1 if x >=1 and x < 11 else 0)
data['basket_1020'] = data['days_in_basket'].apply(lambda x: 1 if x >=11 and x < 20 else 0)
data['basket_2030'] = data['days_in_basket'].apply(lambda x: 1 if x >=21 else 0)

In [37]:
# customer_tpye
data['cust_Tr'] = data['customer_type'].apply(lambda x: 1 if x == 'Transient' else 0)
data['cust_Tr&P'] = data['customer_type'].apply(lambda x: 1 if x == 'Transient-Party' else 0)
data['cust_Con'] = data['customer_type'].apply(lambda x: 1 if x == 'Contract' else 0)
data['cust_Gr'] = data['customer_type'].apply(lambda x: 1 if x == 'Group' else 0)
data = data.drop('customer_type', axis = 1)

In [38]:
# required_parking
data['parking'] = data['required_parking'].apply(lambda x: 1 if x >= 1 else 0)
data = data.drop('required_parking', axis = 1)

In [39]:
# number_of_requests
data['request_0'] = data['number_of_requests'].apply(lambda x: 1 if x == 0 else 0)
data['request_1'] = data['number_of_requests'].apply(lambda x: 1 if x == 1 else 0)
data['request_2'] = data['number_of_requests'].apply(lambda x: 1 if x == 2 else 0)
data['request_3'] = data['number_of_requests'].apply(lambda x: 1 if x == 3 else 0)
data['request_4'] = data['number_of_requests'].apply(lambda x: 1 if x >= 4 else 0)

## Data 이상치

In [40]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 119390 entries, 0 to 119389
Columns: 159 entries, is_canceled to request_4
dtypes: float64(1), int32(2), int64(156)
memory usage: 144.8 MB


In [41]:
data[['lead_time', 'week_number', 'day', 'stays_in_weekend', 'stays_in_week',
     'pre_cancel', 'pre_not_cancel', 'booking_changes']].describe()

,lead_time,week_number,day,stays_in_weekend,stays_in_week,pre_cancel,pre_not_cancel,booking_changes
count,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000
mean,104.011416,27.165173,15.798241,0.927599,2.500302,0.087118,0.137097,0.221124
std,106.863097,13.605138,8.780829,0.998613,1.908286,0.844336,1.497437,0.652306
min,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,18.000000,16.000000,8.000000,0.000000,1.000000,0.000000,0.000000,0.000000
50%,69.000000,28.000000,16.000000,1.000000,2.000000,0.000000,0.000000,0.000000
75%,160.000000,38.000000,23.000000,2.000000,3.000000,0.000000,0.000000,0.000000
max,737.000000,53.000000,31.000000,19.000000,50.000000,26.000000,72.000000,21.000000


# Train and  Test Set

In [42]:
train = data.query('is_canceled != 3')
test = data.query('is_canceled == 3')
test = test.drop('is_canceled', axis = 1)

## Test Set 생성

In [43]:
from sklearn.model_selection import train_test_split

In [44]:
np.random.seed(123)
target = train.is_canceled
data_set = train.drop(columns='is_canceled', axis = 1)
x_train, x_test, y_train, y_test = train_test_split(data_set, target, test_size=0.3,
                                                   stratify=target)

## GridSearch Pipeline

In [45]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score

## xgb

In [46]:
pipe_xgb = Pipeline([('알고리즘',XGBClassifier())])

In [47]:
param_xgb = [
    {'알고리즘':[XGBClassifier()],
     '알고리즘__learning_rate': [0.01, 0.1],
     '알고리즘__n_estimators':[ 900, 1000, 1200, 1500],
     '알고리즘__max_depth': np.arange(17,26)}]

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
xgb = RandomizedSearchCV(pipe_xgb, param_xgb, n_iter=50,
                             scoring='roc_auc',cv=4, n_jobs=-1, verbose=1)
xgb.fit(x_train,y_train)

Fitting 4 folds for each of 50 candidates, totalling 200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


In [ ]:
xgb.best_params_

In [ ]:
pred = xgb.predict_proba(x_test)[:,1]
roc_auc_score(y_test, pred)

## Random Forest

In [ ]:
pipe_rf = Pipeline([('알고리즘',RandomForestClassifier())])

In [ ]:
param_rf = [
    {'알고리즘':[RandomForestClassifier()],
     '알고리즘__max_depth': np.arange(20,31),
     '알고리즘__n_estimators':[800, 900, 1000, 1200, 1500]}]

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
rf = RandomizedSearchCV(pipe_rf, param_rf, n_iter=60,
                             scoring='roc_auc',cv=4, n_jobs=-1, verbose=1)
rf.fit(x_train,y_train)

In [ ]:
rf.best_params_

In [ ]:
pred = rf.predict_proba(x_test)[:,1]
roc_auc_score(y_test, pred)

## Gradient Boost

In [ ]:
pipe_gb = Pipeline([('알고리즘',GradientBoostingClassifier())])

In [ ]:
param_gb = [
    {'알고리즘':[GradientBoostingClassifier()],
     '알고리즘__max_depth': np.arange(5,21),
     '알고리즘__n_estimators':[100, 200, 300, 400, 500]}]

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
gb = RandomizedSearchCV(pipe_gb, param_gb, n_iter=60,
                             scoring='roc_auc',cv=4, n_jobs=-1, verbose=1)
gb.fit(x_train,y_train)

In [ ]:
gb.best_params_

## Voting

### 1. Voting에 사용될 알고리즘 최적의 파라미터로 Fitting

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.preprocessing import StandardScaler

In [ ]:
#xgb = XGBClassifier(n_estimators = 800, max_depth = 18, learning_rate = 0.01, objective='binary:logistic') #standard Scaler
#evals = [(x_test, y_test)]

#rf = RandomForestClassifier(n_estimators = 800, max_depth = 25) # None

#lr = LogisticRegression(C = 1) # Standard Scaler

#dt = DecisionTreeClassifier(max_depth = 12) #Standard Scaler

#ab = AdaBoostClassifier(learning_rate = 1, n_estimators = 800) #Standard Scaler

#kn = KNeighborsClassifier(n_neighbors = 13)

#gb = GradientBoostingClassifier(n_estimators = 200, max_depth = 11) # 미정


In [ ]:
#xgb.fit(x_train, y_train, eval_metric='auc', eval_set=evals,
#        early_stopping_rounds=100, verbose=True)
#rf.fit(x_train, y_train)
#lr.fit(x_train, y_train)
#dt.fit(x_train, y_train)
#ab.fit(x_train, y_train)
#kn.fit(x_train, y_train)
#gb.fit(x_train, y_train)

## RandomForest

In [135]:
rf = RandomForestClassifier(n_estimators = 1500, max_depth = 27 )
rf.fit(x_train, y_train)

RandomForestClassifier(max_depth=27, n_estimators=1500)

In [136]:
pred = rf.predict_proba(x_test)[:,1]
roc_auc_score(y_test, pred)

0.9489057344926393

In [128]:
result['is_canceled'] = rf.predict_proba(test)[:,1]
result.to_csv('rf2.csv', index = False)

## XGBooster

In [97]:
xgb = XGBClassifier(n_estimators = 1000, max_depth = 25, learning_rate = 0.01, objective='binary:logistic') #standard Scaler
evals = [(x_test, y_test)]

In [98]:
xgb.fit(x_train, y_train, eval_metric='auc', eval_set=evals,
        early_stopping_rounds=100, verbose=True)

[0]	validation_0-auc:0.92744
Will train until validation_0-auc hasn't improved in 100 rounds.
[1]	validation_0-auc:0.92839
[2]	validation_0-auc:0.92867
[3]	validation_0-auc:0.92872
[4]	validation_0-auc:0.92865
[5]	validation_0-auc:0.92880
[6]	validation_0-auc:0.92890
[7]	validation_0-auc:0.93022
[8]	validation_0-auc:0.93099
[9]	validation_0-auc:0.93134
[10]	validation_0-auc:0.93191
[11]	validation_0-auc:0.93213
[12]	validation_0-auc:0.93255
[13]	validation_0-auc:0.93288
[14]	validation_0-auc:0.93316
[15]	validation_0-auc:0.93344
[16]	validation_0-auc:0.93360
[17]	validation_0-auc:0.93376
[18]	validation_0-auc:0.93387
[19]	validation_0-auc:0.93402
[20]	validation_0-auc:0.93415
[21]	validation_0-auc:0.93432
[22]	validation_0-auc:0.93451
[23]	validation_0-auc:0.93462
[24]	validation_0-auc:0.93482
[25]	validation_0-auc:0.93491
[26]	validation_0-auc:0.93503
[27]	validation_0-auc:0.93523
[28]	validation_0-auc:0.93536
[29]	validation_0-auc:0.93545
[30]	validation_0-auc:0.93557
[31]	validation

[266]	validation_0-auc:0.94591
[267]	validation_0-auc:0.94592
[268]	validation_0-auc:0.94593
[269]	validation_0-auc:0.94595
[270]	validation_0-auc:0.94597
[271]	validation_0-auc:0.94598
[272]	validation_0-auc:0.94600
[273]	validation_0-auc:0.94601
[274]	validation_0-auc:0.94603
[275]	validation_0-auc:0.94605
[276]	validation_0-auc:0.94606
[277]	validation_0-auc:0.94608
[278]	validation_0-auc:0.94610
[279]	validation_0-auc:0.94611
[280]	validation_0-auc:0.94613
[281]	validation_0-auc:0.94615
[282]	validation_0-auc:0.94617
[283]	validation_0-auc:0.94619
[284]	validation_0-auc:0.94621
[285]	validation_0-auc:0.94622
[286]	validation_0-auc:0.94624
[287]	validation_0-auc:0.94626
[288]	validation_0-auc:0.94628
[289]	validation_0-auc:0.94630
[290]	validation_0-auc:0.94632
[291]	validation_0-auc:0.94634
[292]	validation_0-auc:0.94636
[293]	validation_0-auc:0.94638
[294]	validation_0-auc:0.94639
[295]	validation_0-auc:0.94641
[296]	validation_0-auc:0.94643
[297]	validation_0-auc:0.94644
[298]	va

[531]	validation_0-auc:0.94883
[532]	validation_0-auc:0.94884
[533]	validation_0-auc:0.94884
[534]	validation_0-auc:0.94885
[535]	validation_0-auc:0.94886
[536]	validation_0-auc:0.94886
[537]	validation_0-auc:0.94887
[538]	validation_0-auc:0.94888
[539]	validation_0-auc:0.94888
[540]	validation_0-auc:0.94889
[541]	validation_0-auc:0.94890
[542]	validation_0-auc:0.94890
[543]	validation_0-auc:0.94891
[544]	validation_0-auc:0.94891
[545]	validation_0-auc:0.94892
[546]	validation_0-auc:0.94892
[547]	validation_0-auc:0.94893
[548]	validation_0-auc:0.94893
[549]	validation_0-auc:0.94894
[550]	validation_0-auc:0.94895
[551]	validation_0-auc:0.94896
[552]	validation_0-auc:0.94897
[553]	validation_0-auc:0.94897
[554]	validation_0-auc:0.94897
[555]	validation_0-auc:0.94898
[556]	validation_0-auc:0.94899
[557]	validation_0-auc:0.94900
[558]	validation_0-auc:0.94901
[559]	validation_0-auc:0.94901
[560]	validation_0-auc:0.94901
[561]	validation_0-auc:0.94902
[562]	validation_0-auc:0.94903
[563]	va

[796]	validation_0-auc:0.95000
[797]	validation_0-auc:0.95000
[798]	validation_0-auc:0.95000
[799]	validation_0-auc:0.95001
[800]	validation_0-auc:0.95001
[801]	validation_0-auc:0.95001
[802]	validation_0-auc:0.95002
[803]	validation_0-auc:0.95002
[804]	validation_0-auc:0.95002
[805]	validation_0-auc:0.95002
[806]	validation_0-auc:0.95002
[807]	validation_0-auc:0.95003
[808]	validation_0-auc:0.95003
[809]	validation_0-auc:0.95003
[810]	validation_0-auc:0.95003
[811]	validation_0-auc:0.95004
[812]	validation_0-auc:0.95004
[813]	validation_0-auc:0.95004
[814]	validation_0-auc:0.95005
[815]	validation_0-auc:0.95005
[816]	validation_0-auc:0.95005
[817]	validation_0-auc:0.95005
[818]	validation_0-auc:0.95005
[819]	validation_0-auc:0.95005
[820]	validation_0-auc:0.95005
[821]	validation_0-auc:0.95006
[822]	validation_0-auc:0.95006
[823]	validation_0-auc:0.95006
[824]	validation_0-auc:0.95006
[825]	validation_0-auc:0.95007
[826]	validation_0-auc:0.95007
[827]	validation_0-auc:0.95007
[828]	va

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.01, max_delta_step=0, max_depth=25,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=1000, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [99]:
pred = xgb.predict_proba(x_test)[:,1]
roc_auc_score(y_test, pred)

0.95035584911872

In [100]:
result['is_canceled'] = xgb.predict_proba(test)[:,1]
result.to_csv('xgb3.csv', index = False)

## GradientBoost

In [137]:
gb = GradientBoostingClassifier(n_estimators = 200, max_depth = 11)
gb.fit(x_train, y_train)

GradientBoostingClassifier(max_depth=11, n_estimators=200)

In [138]:
pred = gb.predict_proba(x_test)[:,1]
roc_auc_score(y_test, pred)

0.9500728472453492

In [ ]:
result['is_canceled'] = gb.predict_proba(test)[:,1]
result.to_csv('gb2.csv', index = False)

## Logistic Regression

In [ ]:
lr = LogisticRegression(C = 1)
lr.fit(x_train, y_train)

In [ ]:
pred = lr.predict_proba(x_test)[:,1]
roc_auc_score(y_test, pred)

## DecisionTree

In [ ]:
dt = DecisionTreeClassifier(max_depth = 11)
dt.fit(x_train, y_train)

In [ ]:
pred = dt.predict_proba(x_test)[:,1]
roc_auc_score(y_test, pred)

## AdaBoostClassifier

In [ ]:
ab = AdaBoostClassifier(learning_rate = 1, n_estimators =1000)
ab.fit(x_train, y_train)

In [ ]:
pred = ab.predict_proba(x_test)[:,1]
roc_auc_score(y_test, pred)

## KNeighborsClassifier

In [ ]:
kn = KNeighborsClassifier(n_neighbors = 19)
kn.fit(x_train, y_train)

In [ ]:
pred = kn.predict_proba(x_test)[:,1]
roc_auc_score(y_test, pred)

### 2. Voting 실행

In [139]:
from sklearn.ensemble import VotingClassifier
models = [('XGB', xgb),
          ('RF', rf),
          ('GB', gb)]
hard_vot = VotingClassifier(estimators=models, voting='hard')
soft_vot = VotingClassifier(estimators=models, voting='soft')

#('LR', lr),('DT', dt),('AB', ab),('KNN', kn),

In [ ]:
hard_vot.fit(x_train, y_train)

In [ ]:
soft_vot.fit(x_train, y_train)

### 3. Voting_ ROC 계산

In [ ]:
from sklearn.metrics import roc_auc_score
my_models = [xgb, rf, gb, hard_vot, soft_vot]
for m in my_models:
    try:
        pred = m.predict_proba(x_test)[:,1] # test으로 계산.
    except:
        pred = m.predict(x_test)
    name = m.__class__.__name__ # 이름
    score = roc_auc_score(y_test, pred)
    print('{0} ROC-AUC SCORE:  {1:.4f}'.format(name, score))

In [ ]:
result['is_canceled'] = soft_vot.predict_proba(test)[:,1]
result.to_csv('sof_vot7.csv', index = False)